In [2]:
import numpy as np
import h5py
import pandas as pd
import time

In [3]:
h5_path = '/home/jan/Documents/CodeTDMStoHDF/Ausgangsdaten/examplerRun.h5'
Slice_name = 'Slice00065'
part_name = '0_00003_Canti3_cls'

In [342]:
h5 = h5py.File(h5_path)                                          
Y_Axis = np.array(h5[part_name][Slice_name]['Y-Axis'][:]).astype(int)
Area = np.array(h5[part_name][Slice_name]['Area'][:]).astype(int)
Intensity = np.array(h5[part_name][Slice_name]['Intensity'][:]).astype(int)
X_Axis = np.array(h5[part_name][Slice_name]['X-Axis'][:]).astype(int)
h5.close()

In [343]:
combos = np.stack((X_Axis, Y_Axis, Area, Intensity), axis=-1)

In [344]:
combos

array([[  6039, -19584,      0,      0],
       [  6039, -19584,      0,      0],
       [  6039, -19584,      0,      0],
       ...,
       [  9567, -19584,      0,      0],
       [  9567, -19584,      0,      0],
       [  9567, -19584,      0,      0]])

In [345]:
area_zeros = np.where(combos[:,2]== 0) 
intensity_zeros = np.where(combos[:,3]==0)
zero_area_intensity_indices = np.intersect1d(area_zeros, intensity_zeros)

In [346]:
area_zeros

(array([     0,      1,      2, ..., 142800, 142801, 142802]),)

In [347]:
intensity_zeros

(array([     0,      1,      2, ..., 142800, 142801, 142802]),)

In [348]:
zero_area_intensity_indices

array([     0,      1,      2, ..., 142800, 142801, 142802])

In [349]:
combos_wo_only_zeros = np.delete(combos, zero_area_intensity_indices, axis=0)

In [350]:
combos_wo_only_zeros.shape[0] #all rows with not only Zeros for area and intensity 

81556

In [351]:
combos_wo_only_zeros = np.vstack((combos_wo_only_zeros, [ 10349, -24983,    218,    573]))

In [352]:
xy_combos_wo_only_zeros_unique, unique_indices = np.unique(combos_wo_only_zeros[:,[0,1]],axis=0, return_index = True)

In [353]:
combos_unique = combos_wo_only_zeros[unique_indices]

In [354]:
combos_unique.shape[0]

64117

In [355]:
Index_range = np.arange(combos_wo_only_zeros.shape[0])
indices_of_interest = np.setdiff1d(Index_range, unique_indices) #all the indices belonging to non unique x,y-combinations 

In [356]:
np.unique(combos_wo_only_zeros[indices_of_interest][:,[0,1]],axis=0, return_index = True)

(array([[  9549, -28561],
        [  9549, -28523],
        [  9549, -28507],
        ...,
        [ 10985, -19612],
        [ 10985, -19596],
        [ 10985, -19565]]), array([ 183, 2879, 2880, ..., 3259, 3258,  196]))

In [358]:
indices_of_interest.shape[0]

17440

In [359]:
combos_wo_only_zeros[indices_of_interest]

array([[ 10985, -25033,    171,    536],
       [  9549, -25033,    204,    566],
       [ 10985, -24983,    233,    544],
       ...,
       [  9567, -19584,    199,      0],
       [  9567, -19584,     99,      0],
       [ 10349, -24983,    218,    573]])

In [361]:
combo_processed_array = np.empty([0,4],dtype= int)
start_time = time.time()
combos_wo_only_zeros_copy = np.copy(combos_wo_only_zeros)
index_counter = 0
shape_counter = 0
indices_list = []
for index in indices_of_interest:
    xy_combo = combos_wo_only_zeros[:,[0,1]][index]
    #print(index)
    #index_counter += 1
    #print(xy_combo)
    #print(np.where((combo_processed_array[:,0] == xy_combo[0])*(combo_processed_array[:,1] == xy_combo[1]))[0].size)
    if np.where((combo_processed_array[:,0] == xy_combo[0])*(combo_processed_array[:,1] == xy_combo[1]))[0].size == 0:
        #print('yes')
        #combos_wo_only_zeros[:,[0,1]][index] not in combo_processed_array [:,[0,1]]: #
        index_counter += 1
        xy_combo = combos_wo_only_zeros[:,[0,1]][index]
        #alle indices of this combo
        #print(xy_combo)
        indices_relevant = np.where((combos_wo_only_zeros[:,0] == xy_combo[0])*(combos_wo_only_zeros[:,1] == xy_combo[1]))[0]
        max_area_of_combo = np.amax(combos_wo_only_zeros[:,2][indices_relevant])
        max_intensity_of_combo = np.amax(combos_wo_only_zeros[:,3][indices_relevant])


        #now the values for area and intensity of the array with unique values need to be changed
        #first create a unique array with the unique indices (combos unique)

        #relevant_index_combos_unique = np.where((combos_unique[:,0] == xy_combo[0])*(combos_unique[:,1] == xy_combo[1]))[0]
        #getting array with max values 

        max_combos = np.stack((xy_combo[0], xy_combo[1], max_area_of_combo, max_intensity_of_combo), axis=-1)
        #combos_wo_only_zeros_copy = np.delete(combos_wo_only_zeros_copy, indices_relevant, axis = 0)
        combos_wo_only_zeros_copy = np.vstack((combos_wo_only_zeros_copy, max_combos))
        shape_counter += indices_relevant.shape[0]
        indices_list.append(list(indices_relevant))

        combo_processed_array =  np.vstack((combo_processed_array, max_combos))
        #print(combo_processed_array)

    
indices_relevant = np.hstack(indices_list) 
combos_wo_only_zeros_copy = np.delete(combos_wo_only_zeros_copy, indices_relevant, axis = 0)   
print (time.time()-start_time)
print(index_counter)
print(shape_counter)

11.767669677734375
6643
24083


In [362]:
combos_wo_only_zeros_copy.shape

(64117, 4)

In [336]:
np.hstack(indices_list)

array([ 81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,
        94,   0, 151, 152])

In [339]:
combo_processed_array

array([[ 10985, -24983,    273,    558],
       [ 10349, -24983,    218,    573]])

In [278]:
combos_wo_only_zeros_copy.shape

(138, 4)

In [200]:
combos_wo_only_zeros_copy.shape

(81690, 4)

In [195]:
xy_combos_wo_only_zeros_copy_unique, unique_indices = np.unique(combos_wo_only_zeros_copy[:,[0,1]],axis=0, return_index = True)

In [197]:
xy_combos_wo_only_zeros_copy_unique.shape #vs 64117 

(63945, 2)

In [160]:
combos_wo_only_zeros_copy[:,[0,1]].shape

(80814, 2)

In [147]:
combos_wo_only_zeros_copy = np.copy(combos_wo_only_zeros)

In [156]:
combos_wo_only_zeros_copy.shape

(81547, 4)

In [149]:
combos_wo_only_zeros_copy = np.delete(combos_wo_only_zeros_copy, indices_relevant, axis = 0)

In [155]:
combos_wo_only_zeros_copy = np.vstack((combos_wo_only_zeros_copy, max_combos))

In [124]:
[ 10985, -25033] not in combo_processed_array [:,[0,1]]

False

In [ ]:
#Ziel: 64117 müssen rauskommen 

In [40]:
relevant_index_combos_unique = np.where((combos_unique[:,0] == xy_combo[0])*(combos_unique[:,1] == xy_combo[1]))[0]
relevant_index_combos_unique

array([63045])

In [57]:
combos_unique[[63045]][:,3] = np.asarray([230])

In [167]:
combos_unique[:,3][[63045]]

array([444])

In [133]:
xy_combo = combos_wo_only_zeros[:,[0,1]][1]

In [126]:
indices_relevant = np.where((combos_wo_only_zeros[:,0] == xy_combo[0])*(combos_wo_only_zeros[:,1] == xy_combo[1]))[0]

In [151]:
indices_relevant

array([76157, 76158, 76159, 76160, 76161, 76162, 76163, 76164, 76165,
       76166])

In [70]:
max_area_of_combo = np.amax(combos_wo_only_zeros[:,2][indices_relevant])

In [71]:
max_intensity_of_combo = np.amax(combos_wo_only_zeros[:,3][indices_relevant])

In [134]:
max_combos = np.stack((xy_combo[0], xy_combo[1], max_area_of_combo, max_intensity_of_combo), axis=-1)

In [135]:
max_combos

array([ 10985, -25024,    419,    584])

In [81]:
combo_processed_array = np.empty([0,4],dtype= int)
combo_processed_array

array([], shape=(0, 4), dtype=int64)

In [82]:
combo_processed_array =  np.vstack((combo_processed_array,max_combos))

In [83]:
combo_processed_array

array([[ 10985, -25024,    179,    513]])

In [107]:
np.where((combos_wo_only_zeros[:,0] == xy_combos_wo_only_zeros_unique[2][0])*(combos_wo_only_zeros[:,1] == xy_combos_wo_only_zeros_unique[2][1]))[0].shape

(1,)

In [108]:
time_start = time.time()
for i in range(xy_combos_wo_only_zeros_unique.shape[0]): #über alle Kombinationen ohne Duplikate iterieren
    indices_relevant = np.where((combos_wo_only_zeros[:,0] == xy_combos_wo_only_zeros_unique[i][0])*(combos_wo_only_zeros[:,1] == xy_combos_wo_only_zeros_unique[i][1]))[0]
    if indices_relevant.shape[0] > 1:
        print(indices_relevant)
    
    
    
    
    
print(time.time()-time_start)

#möglich, aber dauert deutlich zu lang, weil zu viele Iterationen 

[ 5517  5518  5519  5520  5521  5522  5523  5524  5525  5526  5527  5528
  5529  5530 31739 31740 31741 31742 31743 31744 31745 31746 31747 31748
 31749 31750 31751 31752]
[ 5076 30175]
[ 5078 30182]
[ 5079 30185]
[ 5080 30189]
[ 5081 30192]
[ 5082 30196]
[ 5083 30199]
[ 5085 30206]
[ 5086 30211]
[ 5089 30221]
[ 5090 30225]
[ 5092 30232]
[ 5093 30235]
[ 5095 30242]
[ 5096 30246]
[ 5099 30256]
[ 5100 30261]
[ 5103 30271]
[ 5104 30275]
[ 5105 30278]
[ 5106 30282]
[ 5107 30285]
[ 5109 30292]
[ 5110  5111 30295 30296]
[ 4865 29449 29450]
[ 4866 29451]
[ 4867 29456]
[ 4870 29466]
[ 4871 29470]
[ 4872  4873 29473 29474]
[ 4473 28088 28089]
[ 4474 28090]
[ 4475 28093]
[ 4477 28100]
[ 4478 28105]
[ 4480  4481 28112 28113]
[ 4082  4083 26727 26728]
[ 4084 26732]
[ 4085 26735]
[ 4086 26739]
[ 4087 26742]
[ 4089 26749]
[ 4090 26751 26752]
[ 3892  3893  3894  3895  3896  3897  3898  3899  3900  3901  3902  3903
 26064 26065 26066 26067 26068 26069 26070 26071 26072 26073 26074 26075
 26076 26077 2

[  973 15865]
[11087 51175]
[66448 66449 66450 66451 66452 66453 66454 66455 66456 66457]
[ 9731 46436]
[ 6582 35477]
[ 6192 34116]
[69885 69886 69887 69888 69889 69890 69891 69892 69893 69894 69895 69896]
[ 4860 29433]
[ 4468 28072]
[ 2926 22679]
[61481 61482 61483 61484 61485 61486 61487 61488 61489 61490 61491 61492]
[ 1744 18560]
[  956 15810]
[  565 14448]
[12073 54606]
[11485 52561]
[10513 49163]
[10123 47800]
[ 9313 45010]
[ 7362 38199]
[64909 64910 64911 64912 64913 64914 64915 64916 64917 64918 64919]
[69113 69114 69115 69116 69117 69118 69119 69120 69121 69122 69123 69124]
[ 4077 26710]
[ 2145 19957]
[  181 13110]
[11868 53899]
[11467 52503]
[ 9506 45670]
[ 8340 41608]
[ 7553 38859]
[ 6609 35541]
[ 6217 34180]
[59949 59950 59951 59952 59953 59954 59955 59956 59957 59958]
[64145 64146 64147 64148 64149 64150 64151 64152 64153 64154 64155 64156]
[12074 54609]
[11069 51117]
[10124 47803]
[ 9732 46440]
[67591 67592 67593 67594 67595 67596 67597 67598 67599 67600]
[ 6191 34112]
[ 

[ 5126 30350]
[ 3533 24790]
[ 2116 19861]
[12427 55865]
[64124 64125 64126 64127 64128 64129 64130 64131 64132 64133 64134 64135]
[12084 54645]
[11059 51081]
[10867 50422]
[10477 49059]
[10134 47839]
[ 9742 46476]
[ 8959 43751]
[ 8369 41704]
[ 7940 40209]
[ 7979 40342]
[ 7008 36940]
[ 6181 34076]
[67571 67572 67573 67574 67575 67576 67577 67578 67579 67580]
[59153 59154 59155 59156 59157 59158 59159 59160 59161 59162 59163]
[ 4457 28032]
[ 4066 26671]
[ 1733 18520]
[ 1344 17157]
[  554 14408]
[11496 52601]
[10524 49203]
[ 9302 44970]
[ 8713 42908]
[ 7351 38159]
[ 6962 36797]
[62601 62602 62603 62604 62605 62606 62607 62608 62609 62610]
[66801 66802 66803 66804 66805 66806 66807 66808 66809 66810 66811]
[ 4889 29531]
[ 3534 24793]
[ 3143 23430]
[ 2897 22581]
[ 2505 21220]
[61841 61842 61843 61844 61845 61846 61847 61848 61849 61850]
[  985 15908]
[65271 65272 65273 65274 65275 65276 65277 65278 65279 65280]
[69473 69474 69475 69476 69477 69478 69479 69480 69481 69482 69483]
[ 5500 31680

[62582 62583 62584 62585 62586 62587 62588 62589 62590]
[66781 66782 66783 66784 66785 66786 66787 66788 66789 66790]
[ 5137 30389]
[ 2556 21395]
[ 2166 20033]
[10856 50383]
[ 8970 43790]
[ 8319 41532]
[ 7929 40170]
[ 7990 40381]
[ 7019 36979]
[ 6630 35617]
[61821 61822 61823 61824 61825 61826 61827 61828 61829 61830]
[ 5490 31645]
[ 4117 26846]
[ 2105 19821]
[  221 13246]
[12416 55825]
[11828 53763]
[10466 49019]
[10074 47656]
[ 9546 45806]
[ 8380 41744]
[ 7593 38995]
[65251 65252 65253 65254 65255 65256 65257 65258 65259 65260]
[69452 69453 69454 69455 69456 69457 69458 69459 69460 69461 69462]
[60295 60296 60297 60298 60299 60300 60301 60302 60303 60304 60305 60306]
[ 4900 29570]
[ 4508 28209]
[ 3154 23469]
[  159 13031]
[64482 64483 64484 64485 64486 64487 64488 64489 64490 64491 64492 64493]
[11890 53978]
[ 9484 45591]
[ 8318 41529]
[ 7531 38780]
[ 6631 35620]
[ 6239 34259]
[67931 67932 67933 67934 67935 67936 67937 67938 67939 67940 67941 67942]
[59511 59512 59513 59514 59515 595

[10456 48983]
[10064 47620]
[ 9556 45842]
[ 8390 41780]
[ 7603 39031]
[65233 65234 65235 65236 65237 65238 65239 65240 65241]
[69431 69432 69433 69434 69435 69436 69437 69438 69439 69440]
[ 4910 29605]
[ 4518 28244]
[ 3464 24570]
[ 3164 23504]
[ 2876 22507]
[ 1794 18732]
[ 1006 15982]
[  615 14620]
[11435 52389]
[60272 60273 60274 60275 60276 60277 60278 60279 60280 60281 60282]
[64460 64461 64462 64463 64464 64465 64466 64467 64468 64469]
[12405 55786]
[10845 50343]
[10455 48980]
[ 8981 43830]
[67908 67909 67910 67911 67912 67913 67914 67915 67916 67917 67918]
[ 8391 41783]
[ 7918 40130]
[ 8001 40421]
[ 7030 37019]
[ 3555 24869]
[ 2484 21144]
[ 2094 19782]
[59488 59489 59490 59491 59492 59493 59494 59495 59496 59497 59498 59499]
[62935 62936 62937 62938 62939 62940 62941 62942 62943 62944 62945]
[67144 67145 67146 67147 67148 67149 67150 67151 67152 67153 67154 67155]
[ 3854 25930]
[ 3463 24567]
[ 1711 18441]
[ 1795 18735]
[ 1322 17078]
[ 1406 17372]
[  532 14329]
[  616 14623]
[11434

...

[   49 12638]
[12000 54371]
[ 9374 45198]
[ 8208 41136]
[ 7421 38387]
[ 6741 36013]
[ 6349 34652]
[59666 59667 59668 59669 59670 59671 59672 59673 59674 59675]
[63859 63860 63861 63862 63863 63864 63865 63866 63867 63868 63869 63870]
[ 5248 30786]
[ 3655 25226]
[ 2384 20787]
[ 1994 19425]
[12305 55429]
[10745 49986]
[10355 48623]
[ 9081 44187]
[67311 67312 67313 67314 67315 67316 67317 67318 67319 67320]
[ 8491 42140]
[ 7818 39773]
[ 8101 40778]
[ 7130 37376]
[58898 58899 58900 58901 58902 58903 58904 58905 58906 58907 58908]
[ 4335 27596]
[ 3944 26235]
[ 1611 18084]
[ 1222 16721]
[62341 62342 62343 62344 62345 62346 62347 62348 62349 62350 62351 62352]
[  432 13972]
[11618 53037]
[10646 49639]
[66545 66546 66547 66548 66549 66550 66551 66552 66553 66554 66555]
[ 9180 44534]
[ 8591 42472]
[ 7229 37723]
[ 6840 36361]
[ 5249 30789]
[ 2668 21795]
[61580 61581 61582 61583 61584 61585 61586 61587 61588 61589]
[ 2278 20433]
[10744 49983]
[ 9082 44190]
[ 8207 41132]
[ 7817 39770]
[ 8102 40781

[66525 66526 66527 66528 66529 66530 66531 66532 66533 66534]
[ 5259 30825]
[ 2678 21831]
[ 2288 20469]
[10734 49947]
[ 9092 44226]
[ 8197 41096]
[ 7807 39734]
[ 8112 40817]
[ 7141 37415]
[ 6752 36053]
[61561 61562 61563 61564 61565 61566 61567 61568 61569]
[64992 64993 64994 64995 64996 64997 64998 64999 65000 65001 65002 65003]
[ 5368 31209]
[ 4239 27282]
[ 1983 19385]
[  343 13682]
[12294 55389]
[11706 53327]
[10344 48583]
[ 9952 47220]
[ 9668 46242]
[ 8502 42180]
[ 7715 39431]
[69183 69184 69185 69186 69187 69188 69189 69190 69191 69192 69193]
[60027 60028 60029 60030 60031 60032 60033 60034 60035 60036 60037 60038]
[64218 64219 64220 64221 64222 64223 64224 64225 64226 64227 64228]
[ 5022 30005]
[ 4630 28644]
[ 3933 26195]
[ 3352 24170]
[ 3276 23904]
[ 2764 22107]
[ 2289 20472]
[ 1906 19132]
[ 1211 16681]
[ 1118 16382]
[  727 15020]
[11323 51989]
[11232 51692]
[ 8580 42432]
[ 6829 36321]
[ 6437 34960]
[67666 67667 67668 67669 67670 67671 67672 67673 67674 67675 67676 67677]
[59251

[ 5032 30041]
[ 4640 28680]
[60004 60005 60006 60007 60008 60009 60010 60011 60012 60013 60014]
[ 3342 24134]
[ 3286 23940]
[ 2754 22071]
[ 1916 19168]
[ 1128 16418]
[  737 15056]
[64197 64198 64199 64200 64201 64202 64203 64204 64205 64206]
[11313 51953]
[67643 67644 67645 67646 67647 67648 67649 67650 67651 67652 67653]
[ 5270 30865]
[ 4705 28880]
[59229 59230 59231 59232 59233 59234 59235 59236 59237 59238 59239]
[ 3081 23232]
[ 2689 21871]
[  801 15257]
[12283 55350]
[12228 55159]
[10915 50567]
[10723 49907]
[10333 48544]
[10278 48353]
[ 9886 46990]
[ 9103 44266]
[ 8513 42219]
[ 7796 39694]
[ 8123 40857]
[ 7152 37455]
[ 6037 33562]
[62670 62671 62672 62673 62674 62675 62676 62677 62678 62679 62680]
[66877 66878 66879 66880 66881 66882 66883 66884 66885 66886 66887 66888]
[ 5357 31169]
[ 4641 28683]
[ 4250 27322]
[   26 12557]
[  354 13722]
[11695 53287]
[12023 54452]
[ 9941 47180]
[ 9351 45117]
[ 9679 46282]
[ 7398 38306]
[ 7726 39471]
[ 6372 34733]
[61908 61909 61910 61911 61912 6

[ 1140 16461]
[  749 15099]
[11301 51910]
[11254 51771]
[ 8558 42353]
[ 6807 36242]
[ 6415 34881]
[63029 63030 63031 63032 63033 63034 63035 63036 63037 63038 63039]
[67231 67232 67233 67234 67235 67236 67237 67238 67239 67240 67241 67242]
[ 4693 28837]
[ 3093 23275]
[ 2701 21914]
[  789 15214]
[12240 55202]
[10290 48396]
[ 9898 47033]
[ 6025 33519]
[62259 62260 62261 62262 62263 62264 62265 62266 62267 62268 62269]
[ 4653 28726]
[ 4262 27365]
[ 3720 25451]
[ 3329 24088]
[ 1929 19214]
[ 1540 17851]
[  750 15102]
[11300 51907]
[65696 65697 65698 65699 65700 65701 65702 65703 65704 65705 65706]
[ 8909 43602]
[ 5850 32901]
[69925 69926 69927 69928 69929 69930 69931 69932 69933 69934]
[60733 60734 60735 60736 60737 60738 60739 60740 60741 60742 60743]
[ 1141 16465]
[11255 51775]
[ 9899 47036]
[ 6414 34877]
[ 6024 33516]
[64931 64932 64933 64934 64935 64936 64937 64938 64939 64940 64941 64942]
[68378 68379 68380 68381 68382 68383 68384 68385 68386 68387]
[ 5344 31123]
[ 4692 28833]
[ 4300 2

[ 4278 27419]
[ 4273  4274 27404]
[ 3708  3709 25412]
[ 3701 25388]
[ 3317  3318 24049]
[ 3312 24030]
[ 3310 24025]
[ 3112  3113  3114  3115  3116  3117  3118  3119  3120  3121  3122  3123
 23339 23340 23341 23342 23343 23344 23345 23346 23347 23348 23349 23350
 23351 23352]
[ 3111 23336]
[ 3110 23333]
[ 3109 23329]
[ 3106 23319]
[ 3104  3105 23315 23316]
[ 2720  2721  2722  2723  2724  2725  2726  2727  2728  2729  2730  2731
  2732  2733 21978 21979 21980 21981 21982 21983 21984 21985 21986 21987
 21988 21989 21990]
[ 2719 21975]
[ 2717 21968]
[ 2716 21964]
[ 2713 21954 21955]
[ 2330  2331  2332  2333  2334  2335  2336  2337  2338  2339  2340  2341
  2342 20616 20617 20618 20619 20620 20621 20622 20623 20624 20625 20626
 20627 20628]
[ 2329 20612]
[ 2326 20602]
[ 2323 20592 20593]
[ 1565  1566 17923 17924]
[ 1564 17920]
[ 1940  1941  1942  1943  1944  1945  1946  1947  1948  1949  1950  1951
  1952 19253 19254 19255 19256 19257 19258 19259 19260 19261 19262 19263
 19264 19265 19266]


In [44]:
ex_array = np.asarray([10985, -25024,     70,    444])

In [47]:
ex_array[2] = 100

In [48]:
ex_array

array([ 10985, -25024,    100,    444])

In [91]:
np.unique([3, 3, 2, 2, 1, 1])


array([1, 2, 3])

In [229]:
# Problem wahrscheinlich: falsches Verständnis von np unique
ex_array = np.asarray([[3,1], [3,1], [2, 2], [2, 2], [1, 1],[1,1],[1,0]])


In [230]:
ex_array_unique, unique_array = np.unique(ex_array, axis = 0, return_index = True)

In [231]:
ex_array[unique_array]

array([[1, 0],
       [1, 1],
       [2, 2],
       [3, 1]])

In [232]:
Index_range = np.arange(ex_array.shape[0])
indices_of_interest = np.setdiff1d(Index_range, unique_array) #all the indices belonging to non unique x,y-combinations

In [233]:
ex_array[indices_of_interest] #-> es ist klar, dass aus den 

array([[3, 1],
       [2, 2],
       [1, 1]])